# Notebook 1: Introdução à Detecção de Outliers com PyOD (KNN)

## Objetivo
Este notebook introdutório tem como objetivos:
1.  Apresentar o conceito de Detecção de Outliers no contexto de auditoria.
2.  Introduzir a biblioteca **PyOD** (Python Outlier Detection).
3.  Demonstrar o algoritmo **K-Nearest Neighbors (KNN)** para identificar anomalias em dados simulados de despesas.

## Cenário de Auditoria
Imagine que você está auditando despesas de viagens corporativas. A maioria das despesas segue um padrão (valor razoável, distância lógica). No entanto, algumas despesas podem ser fraudulentas ou erros de lançamento (ex: valor muito alto para uma distância curta).

Detectaremos esses casos anômalos.

In [ ]:
# Instalação da biblioteca PyOD (caso não esteja instalada)
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyod.models.knn import KNN
from pyod.utils.data import generate_data

# Configuração de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Geração de Dados Simulados
Vamos criar um dataset sintético que represente **Valor da Despesa** vs **Distância da Viagem**.
- **Dados Normais**: Seguem uma correlação linear positiva (mais longe = mais caro).
- **Anomalias**: Valores fora desse padrão (ex: muito caro para perto).

In [ ]:
contamination = 0.05  # 5% de anomalias esperadas
n_train = 500
n_test = 200

# Gerando dados sintéticos com PyOD
X_train, X_test, y_train, y_test = generate_data(
    n_train=n_train, 
    n_test=n_test, 
    n_features=2, 
    contamination=contamination, 
    random_state=42
)

# Convertendo para DataFrame para facilitar manipulação
df_train = pd.DataFrame(X_train, columns=['Distancia_Viagem', 'Valor_Despesa'])
df_test = pd.DataFrame(X_test, columns=['Distancia_Viagem', 'Valor_Despesa'])

print(f"Dados de Treino: {X_train.shape}")
print(f"Dados de Teste: {X_test.shape}")

### Visualizando os Dados
Vamos plotar os dados para entender a distribuição. Note que `generate_data` já nos dá o 'ground truth' (quais são outliers reais), mas na vida real não saberíamos.

In [ ]:
plt.scatter(df_train['Distancia_Viagem'], df_train['Valor_Despesa'], c='blue', alpha=0.6, label='Dados')
plt.title('Distribuição de Despesas de Viagem (Simulação)')
plt.xlabel('Distância da Viagem (normalizada)')
plt.ylabel('Valor da Despesa (normalizado)')
plt.legend()
plt.show()

## 2. Aplicando o Modelo KNN (K-Nearest Neighbors)
O KNN para anomalias funciona medindo a distância de um ponto para seus vizinhos mais próximos. 
- Pontos **isolados** (longe dos vizinhos) terão scores de anomalia altos.
- Pontos **agrupados** (densos) terão scores baixos.

In [ ]:
# 1. Inicializar o modelo
# method='largest': usa a distância para o k-ésimo vizinho como score
clf_knn = KNN(n_neighbors=5, method='largest', contamination=contamination)

# 2. Treinar o modelo (Fit)
clf_knn.fit(X_train)

# 3. Obter as previsões e scores para os dados de treino
# labels_: 0 para normal, 1 para outlier (binário)
# decision_scores_: valor numérico cru da anomalia (quanto maior, mais anômalo)
y_train_pred = clf_knn.labels_
y_train_scores = clf_knn.decision_scores_

## 3. Analisando os Resultados
Vamos ver quantos outliers o modelo detectou nos dados de treino.

In [ ]:
n_outliers = np.count_nonzero(y_train_pred == 1)
n_inliers = np.count_nonzero(y_train_pred == 0)

print(f"Total de registros: {len(y_train_pred)}")
print(f"Outliers detectados: {n_outliers}")
print(f"Inliers (Normais): {n_inliers}")

### Visualização dos Outliers Detectados

In [ ]:
plt.figure(figsize=(12, 7))

# Plotar inliers (blue)
mask_inliers = y_train_pred == 0
plt.scatter(X_train[mask_inliers, 0], X_train[mask_inliers, 1], 
            c='blue', label='Normal', alpha=0.5, s=20)

# Plotar outliers (red)
mask_outliers = y_train_pred == 1
plt.scatter(X_train[mask_outliers, 0], X_train[mask_outliers, 1], 
            c='red', label='Outlier Detectado', marker='x', s=100)

plt.title('Detecção de Anomalias com KNN')
plt.xlabel('Distância')
plt.ylabel('Valor')
plt.legend()
plt.show()

## 4. Avaliação (Dados de Teste)
Agora vamos aplicar o modelo treinado em novos dados (teste) e verificar a performance, já que neste exemplo sintético possuímos o gabarito (`y_test`).

In [ ]:
from pyod.utils.data import evaluate_print

# Predição nos dados de teste
y_test_pred = clf_knn.predict(X_test)  # labels 0 ou 1
y_test_scores = clf_knn.decision_function(X_test)  # scores brutos

# Avaliação automática do PyOD
print("\nAvaliação no Conjunto de Teste:")
evaluate_print('KNN', y_test, y_test_scores)

In [ ]:
# Verificando o 'Top 5' mais anômalos no teste
df_results = df_test.copy()
df_results['Score_Anomalia'] = y_test_scores
df_results['Eh_Outlier'] = y_test_pred

print("Top 5 transações mais suspeitas:")
display(df_results.sort_values('Score_Anomalia', ascending=False).head())

## Conclusão
O KNN é um método intuitivo e eficaz para detectar anomalias baseadas em distância. Se um ponto está longe de todos os outros pontos "dados", ele é provável que seja uma anomalia.

**Próximos passos:** No próximo notebook, veremos o algoritmo **Isolation Forest**, muito usado para grandes volumes de dados.